# Reading data from hbase using spark

In [1]:
import findspark
import multiprocessing
from pyspark.sql import SparkSession
from pyspark import SQLContext, SparkContext, SparkConf

findspark.init('/home/martin96/Spark/spark-3.2.1-bin-hadoop3.2')
findspark.find()

def read_hbase():
    conf = (SparkConf().setAppName("RW_from_HBase"))

    spark = SparkSession.builder.appName(" ").config(conf=conf).getOrCreate() 

    df = spark.read.format("org.apache.hadoop.hbase.spark").option("hbase.columns.mapping",
            "rowKey STRING :key," + 
            "f_name STRING cf:first_name").option("hbase.spark.pushdown.columnfilter", True) \
                .option("hbase.table", "test_v1") \
                .option("hbase.spark.use.hbasecontext", False) \
                .load()
    df.show()

read_hbase()

22/03/09 21:13:27 WARN Utils: Your hostname, martin96-TP300LA resolves to a loopback address: 127.0.1.1; using 192.168.1.109 instead (on interface wlp2s0)
22/03/09 21:13:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/09 21:13:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


rowKey STRING :key,f_name STRING cf:first_name


+------+------+
|f_name|rowKey|
+------+------+
|Martin|     1|
+------+------+



In [9]:
import findspark
import multiprocessing
from pyspark.sql import SparkSession
from pyspark import SQLContext, SparkContext, SparkConf

findspark.init('/home/martin96/Spark/spark-3.2.1-bin-hadoop3.2')
findspark.find()

def write_to_hbase():
    conf = (SparkConf().setAppName("RW_from_HBase"))

    spark = SparkSession.builder.appName(" ").config(conf=conf).getOrCreate()

    df = spark.read.format("org.apache.hadoop.hbase.spark").option("hbase.columns.mapping",
            "rowKey STRING :key," + 
            "f_name STRING cf:first_name," + "l_name STRING cf:last_name," + "m_name STRING cf:first_name").option("hbase.spark.pushdown.columnfilter", True) \
                .option("hbase.table", "test_v1") \
                .option("hbase.spark.use.hbasecontext", False) \
                .load()

    df.write.format("org.apache.hadoop.hbase.spark").option("hbase.columns.mapping",
            "rowKey STRING :key," + 
            "f_name STRING cf:first_name," + "l_name STRING cf:first_name," + "m_name STRING cf:first_name").option("hbase.spark.pushdown.columnfilter", True) \
                .option("hbase.table", "test_v1") \
                .option("hbase.spark.use.hbasecontext", False) \
                .save()
    df.createOrReplaceTempView("table")
    spark.sql("INSERT INTO table (rowKey, f_name, l_name, m_name) VALUES ('3', 'steve','smith', 'john')")
    df.show()

write_to_hbase()

rowKey STRING :key,f_name STRING cf:first_name,l_name STRING cf:last_name,m_name STRING cf:first_name
rowKey STRING :key,f_name STRING cf:first_name,l_name STRING cf:first_name,m_name STRING cf:first_name


22/03/10 19:41:16 WARN FileOutputCommitter: Output Path is null in setupJob()
22/03/10 19:41:17 WARN FileOutputCommitter: Output Path is null in commitJob()
22/03/10 19:41:17 WARN FileOutputCommitter: Output Path is null in setupJob()
22/03/10 19:41:17 WARN FileOutputCommitter: Output Path is null in commitJob()


+--------+--------+--------+------+
|  f_name|  l_name|  m_name|rowKey|
+--------+--------+--------+------+
|  Martin|    null|  Martin|     1|
|anderson|anderson|anderson|     2|
|    john|   smith|    john|     3|
+--------+--------+--------+------+

